In [1]:
from ga_api_package.ga_api_client import Repository, System, Tenant
import logging
import pandas as pd

# logging.basicConfig(level=logging.INFO)

In [2]:
# create repository object
repo = Repository('https://rdlab-214.genie-analytics.com/api', 'api', 'default', 'api123!@#', ssl=False, burst=1, retry=0)
# create dataset
dset = await repo.set_dataset(13579, {
    'name':'example2',
    'series':'hour',
    'freq':300,
    'retainDepth':86400*7,
    'retainSize':10000000,
    'run':False
})
# delete all pipelines
await dset.del_all_pipelines()
# create pipeline
pipeline = {
    'scope': {
        'i-field': '@controller.home',
        'i-entry': [
            [True,'t:0:0:1',0]
        ],
        'mode': '%directional'
    },
    # 'bucket': [
    #     ['$distinctTuple', {
    #         'fields':['@flow.protocol'],
    #         'unwind':[False],
    #         'sizeLimit':256,
    #         'sortedBy':['$sum', {
    #             'field':'@flow.bytes'
    #         }],
    #         'sortedDir':1,
    #         'nullOpt':False
    #     }]
    # ],
    'metric': [
        ['$sum', {
            'field':'@flow.bytes'
        }]
    ]
}
pipe = await dset.set_pipeline(1001, {
    'name':'example',
    'pipeline': pipeline
})
# specify the time range
freq = '5T'
end = pd.Timestamp.now('+08:00').floor(freq) - pd.Timedelta(freq)
start = end - pd.Timedelta('48H')
dts = pd.date_range(start=start, end=end, freq=freq)

In [3]:
# patch dataset data
df_patch = await dset.patch_data(dts)
df_patch

,1001
timestamp,
2022-03-15 17:25:00+08:00,1
2022-03-15 17:30:00+08:00,1
2022-03-15 17:35:00+08:00,1
2022-03-15 17:40:00+08:00,1
2022-03-15 17:45:00+08:00,1
...,...
2022-03-17 17:05:00+08:00,1
2022-03-17 17:10:00+08:00,1
2022-03-17 17:15:00+08:00,1


In [4]:
df_patch[df_patch.isna().any(axis=1)]

,1001
timestamp,


In [5]:
# poll dataset data
df_poll = await dset.poll_data(dts)
df_poll

,1001
timestamp,
2022-03-15 17:25:00+08:00,1
2022-03-15 17:30:00+08:00,1
2022-03-15 17:35:00+08:00,1
2022-03-15 17:40:00+08:00,1
2022-03-15 17:45:00+08:00,1
...,...
2022-03-17 17:05:00+08:00,1
2022-03-17 17:10:00+08:00,1
2022-03-17 17:15:00+08:00,1


In [6]:
df_poll[df_poll.isna().any(axis=1)]

,1001
timestamp,


In [12]:
# query pipeline data
df_query = await pipe.read_data(dts)
df_query

,0,1,2
0,2022-03-15 17:25:00+08:00,4089303217,775994466
1,2022-03-15 17:30:00+08:00,2683886607,528544784
2,2022-03-15 17:35:00+08:00,45797596,430923866
3,2022-03-15 17:40:00+08:00,61687720,734718231
4,2022-03-15 17:45:00+08:00,59340303,516164426
...,...,...,...
572,2022-03-17 17:05:00+08:00,49920658,771188137
573,2022-03-17 17:10:00+08:00,63446798,1140573695
574,2022-03-17 17:15:00+08:00,67045477,604068074
575,2022-03-17 17:20:00+08:00,70750693,595504911


In [9]:
df_query[df_query.isna().any(axis=1)]

,0,1,2


In [13]:
adhoc = await repo.set_adhoc(300, pipeline)

In [14]:
df_adhoc = await adhoc.read_data(dts, 'hour')
df_adhoc

,0,1,2
0,2022-03-15 17:25:00+08:00,4089303217,775994466
1,2022-03-15 17:30:00+08:00,2683886607,528544784
2,2022-03-15 17:35:00+08:00,45797596,430923866
3,2022-03-15 17:40:00+08:00,61687720,734718231
4,2022-03-15 17:45:00+08:00,59340303,516164426
...,...,...,...
572,2022-03-17 17:05:00+08:00,49920658,771188137
573,2022-03-17 17:10:00+08:00,63446798,1140573695
574,2022-03-17 17:15:00+08:00,67045477,604068074
575,2022-03-17 17:20:00+08:00,70750693,595504911


In [15]:
df_adhoc[df_adhoc.isna().any(axis=1)]

,0,1,2


In [16]:
await repo.close()